In [4]:
# 引用包
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pymysql
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False

In [5]:
# 读取sql数据
con = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='lxy021527', db='ABTest', use_unicode=True, charset='utf8')
data = pd.read_sql('select * from abtest', con=con)
data.head()

,id,组,曝光量,点击量,下单量,购买店铺数,店铺类型,riqi
0,17437864,A,105,16,15,20,A,2020-05-13
1,17402857,A,147,21,13,7,A,2020-05-13
2,17017891,A,59,10,9,4,A,2020-05-13
3,15792190,A,216,39,6,2,C,2020-05-13
4,8726788,A,51,9,6,1,A,2020-05-13


$n_A$ = $kn_B$ and $n_B$ = (1+1/k)(σ$(z_{1-α/2}$+$z_{1-β})$/$(μ_A-μ_B))^2$

In [6]:
# 根据常规默认值确定α，β，K值，α=0.05， β=0.2， K=1(组间样本均值)
alpha = 0.05
beta = 0.2
k = 1

求 $z_{1-α/2}$ 和 $z_{1-β}$

In [7]:
z_alpha = stats.norm.ppf(1-alpha/2)
z_beta = stats.norm.ppf(1-beta)
display(z_alpha, z_beta)

1.959963984540054

0.8416212335729143

In [8]:
from scipy import stats
alpha = 0.05
z = stats.norm.ppf(1-alpha)
z

1.6448536269514722

求$σ$

In [9]:
df_C = data.loc[(data.组=='D') & (data.店铺类型 == 'C'), ['riqi', '下单量']]
sigema = df_C['下单量'].std()
sigema

0.4560376923361705

求二类目标提升值：$μ_A -μ_B$

In [11]:
mean = df_C.groupby('riqi')['下单量'].mean().mean()*0.3
mean

0.031065555555555555

In [13]:
yangbenliang = (1+1/k)*np.power(sigema*(z_alpha+z_beta)/mean, 2)
yangbenliang

3382.83565731299

求σ

In [17]:
df_A = data.loc[(data.组=='D') & (data.店铺类型=='A'), ['riqi','下单量']]
sigema = df_A['下单量'].std()
sigema

1.903068855290103

求一类目标提升值：$μ_A -μ_B$

In [21]:
mean = df_A.groupby('riqi')['下单量'].mean().std()*2
mean

0.19578164968724276

In [22]:
yangbenliang = (1+1/k)*np.power(sigema*(z_alpha+z_beta)/mean, 2)  # 随着$μ_A-μ_B$的插值愈大，所需样本量愈小
yangbenliang

1483.2102112429511